### ¿Qué es LangChain?

**LangChain** es un `framework` de código abierto diseñado para facilitar el desarrollo de aplicaciones que utilizan modelos de lenguaje. Permite unificar la interfaz de acceso a distintos proveedores.

### Características principales LangChain

- **Orquestación modular:** LangChain proporciona componentes modulares que se pueden combinar para crear flujos de trabajo complejos, llamados "cadenas" (pipelines). Cada cadena es una secuencia de pasos que pueden incluir llamadas a modelos de lenguaje, consultas a bases de datos, procesamiento de texto.
- **Integración sencilla:** Permite integrar casi cualquier modelo de lenguaje, tanto de código abierto como comercial, usando una interfaz estándar y sencilla.
- **Gestión de contexto y memoria:** Facilita la gestión del estado de la conversación y el contexto, permitiendo que las aplicaciones recuerden interacciones anteriores y ofrezcan respuestas más coherentes y personalizadas.
- **Automatización y agentes:** Permite crear agentes inteligentes que pueden tomar decisiones, consultar diferentes fuentes de datos y ejecutar acciones de forma autónoma.
- **Soporte para Python y JavaScript:** Está disponible principalmente para estos lenguajes, facilitando su adopción en proyectos modernos.



### RAG con LangChain

Esta práctica es idéntica a la anterior pero adaptada a LangChain. Empezamos cargando el `dataset`.

In [2]:
from google.colab import userdata
import os

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [3]:
!kaggle datasets download -d kotartemiy/topic-labeled-news-dataset

Dataset URL: https://www.kaggle.com/datasets/kotartemiy/topic-labeled-news-dataset
License(s): CC0-1.0
  0% 0.00/9.45M [00:00<?, ?B/s]
100% 9.45M/9.45M [00:00<00:00, 1.02GB/s]


In [4]:
import zipfile

# Define the path to your zip file
file_path = '/content/topic-labeled-news-dataset.zip'

with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/datasets')

In [6]:
import pandas as pd

df = pd.read_csv('/content/datasets/labelled_newscatcher_dataset.csv', sep=';')

In [7]:
MAX_NEWS = 1000
DOCUMENT="title"
TOPIC="topic"

subset_news = df.head(MAX_NEWS)

Aunque hemos leído en `dataset` en Pandas, LangChain puede cargar directamente el fichero `csv` con la librería `document_loader` y cargarlo en ChromaDB:

In [12]:
!pip install -q langchain
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [21]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

Creamos el`loader`, indicando la fuente de datos y el nombre de la columna en el `dataframe` que contiene la información.

In [14]:
df_loader = DataFrameLoader(subset_news, page_content_column=DOCUMENT)

Cargamos y mostramos el documento. Se observa que usa como `metadata` el resto de campos.

In [15]:
df_document = df_loader.load()
display(df_document[:2])

[Document(metadata={'topic': 'SCIENCE', 'link': 'https://www.eurekalert.org/pub_releases/2020-08/dbnl-acl080620.php', 'domain': 'eurekalert.org', 'published_date': '2020-08-06 13:59:45', 'lang': 'en'}, page_content="A closer look at water-splitting's solar fuel potential"),
 Document(metadata={'topic': 'SCIENCE', 'link': 'https://www.pulse.ng/news/world/an-irresistible-scent-makes-locusts-swarm-study-finds/jy784jw', 'domain': 'pulse.ng', 'published_date': '2020-08-12 15:14:19', 'lang': 'en'}, page_content='An irresistible scent makes locusts swarm, study finds')]

Ahora generamos los embeddings. Para ello, será necesario importar **CharacterTextSplitter:** para agrupar la información en `chunks`.


In [16]:
from langchain.text_splitter import CharacterTextSplitter

No existe una forma 100% correcta de dividir los documentos en chunks). La clave está en equilibrar el contexto y el uso de memoria:

- **Fragmentos más grandes:** Proporcionan al modelo más contexto, lo que puede llevar a una mejor comprensión y respuestas más precisas. Sin embargo, consumen más memoria.
- **Fragmentos más pequeños:** Reducen el uso de memoria, pero pueden limitar la comprensión contextual del modelo si la información queda demasiado fragmentada.

Se ha decidido usar un tamaño medio de 250 caracteres para cada `chunk` con un `overloap` de 10 caracteres. Es decir, los 10 caracteres finales de un `chunk`, serán los 10 primeros del siguiente.


In [18]:
text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=10)
texts = text_splitter.split_documents(df_document)
display(texts[:2])

[Document(metadata={'topic': 'SCIENCE', 'link': 'https://www.eurekalert.org/pub_releases/2020-08/dbnl-acl080620.php', 'domain': 'eurekalert.org', 'published_date': '2020-08-06 13:59:45', 'lang': 'en'}, page_content="A closer look at water-splitting's solar fuel potential"),
 Document(metadata={'topic': 'SCIENCE', 'link': 'https://www.pulse.ng/news/world/an-irresistible-scent-makes-locusts-swarm-study-finds/jy784jw', 'domain': 'pulse.ng', 'published_date': '2020-08-12 15:14:19', 'lang': 'en'}, page_content='An irresistible scent makes locusts swarm, study finds')]

Ahora creamos los `embeddings`. Se puede usar directamente LangChain para hacer esto.

In [19]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-19-5ec81d8b8ae6>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creamos la base de datos. Esta instrucción también crea los índices.

In [23]:
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 14.0 MB/s eta

In [24]:
chroma_db = Chroma.from_documents(
    texts, embedding_function
)

El siguiente paso es especificar el `retriever`, que recupera información de los documentos que le proporcionemos. En este caso hace una búsqueda por proximidad de los `embbeddings` almacenados en ChromaDB. El último paso es seleccionar el modelo de lenguaje que recibirá la `pipeline` de Hugging Face.

In [25]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

In [26]:
retriever = chroma_db.as_retriever()

In [42]:
model_id = "google/flan-t5-large"
task="text2text-generation"

hf_llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task=task,
    device_map="auto",
    pipeline_kwargs={
        "max_new_tokens": 256,
        "repetition_penalty":1.1, # penaliza que el modelo repita respuestas en el prompt. Parece que algunos modeos lo hacen
    },
)

Device set to use cuda:0


Ahora configuramos la  `pipeline`:

In [51]:
document_qa = RetrievalQA.from_chain_type(
    llm=hf_llm, retriever=retriever, chain_type='stuff'
)

`chain_type` puede tener los siguientes valores:

- **stuff:** La opción más sencilla; simplemente toma los documentos que considera apropiados y los utiliza en el prompt que se pasa al modelo.
- **refine:** Realiza múltiples llamadas al modelo con diferentes documentos, intentando obtener una respuesta más refinada cada vez. Puede ejecutar un número elevado de llamadas al modelo, por lo que debe usarse con precaución.
- **map_reduce:** Intenta reducir todos los documentos en uno solo, posiblemente a través de varias iteraciones. Puede comprimir y condensar los documentos para que quepan en el prompt enviado al modelo.
- **map_rerank:** Llama al modelo para cada documento y los clasifica, devolviendo finalmente el mejor. Similar a refine, puede ser arriesgado dependiendo del número de llamadas que se prevea realizar.


Ahora, podemos hacer la pregunta:

In [54]:
response = document_qa.invoke("Can I buy a Toshiba laptop?")

display(response)

{'query': 'Can I buy a Toshiba laptop?', 'result': 'no'}

La respuesta es correcta. No se obtiene mucha información porque el modelo usado, T5, no está específicamente preparado para la generación de texto.

In [53]:
response = document_qa.invoke("Can I buy a Acer 3 laptop?")

display(response)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'query': 'Can I buy a Acer 3 laptop?', 'result': 'Yes'}

### LCEL (LangChain Expression Language)

LangChain es un `framework` muy nuevo en constante evolución. LCEL es una nueva sintaxs.

In [55]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [62]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | hf_llm
    | StrOutputParser()
)

In [58]:
chain.invoke("Can I buy a Toshiba laptop?")

"[Document(metadata='lang': 'en', 'domain': 'techweez.com', 'link': 'https://techweez.com/2020/08/10/toshiba-done-with-making-laptops/', 'published_date': '2020-08-10 12:00:13', 'topic': 'TECHNOLOGY', 'domain': 'myjoyonline.com', 'page_content='The Legendary Toshiba is Officially Done With Making Laptops'], 'Toshiba shuts the lid on laptops after 35 years'], 'Apple's Next MacBook Could Be the Cheapest in Company's History'], 'Apple to Reportedly Launch Its Cheapest MacBook Ever']"